# Quantum XOR



In [ ]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import AdamOptimizer
import matplotlib.pyplot as plt

In [ ]:
# generate noisy XOR data
def gen_xor_data(num_samples,noise=2e-1):

    x = np.random.randint(0,2,size=(num_samples,2))
    x_noise = x + np.random.randn(num_samples,2)*2e-1

    y = np.ones((num_samples,1))
    for ck in range(num_samples):
        y[ck] = (x[ck,0] or x[ck,1]) and not(x[ck,0] and x[ck,1])
        
    return x_noise,y

num_samples = 32
x_noise,y = gen_xor_data(num_samples)
x_val_noise, y_val = gen_xor_data(num_samples,noise=3e-1)

In [ ]:
# check the data

colors = np.ones((num_samples,3))
colors[:,2] = y[:,0]
colors[:,0] = 1-y[:,0]

plt.scatter(x_noise[:,0],x_noise[:,1],c=colors,edgecolors=[0.,0.,0.])
plt.show()

# check the data

colors = np.ones((num_samples,3))
colors[:,2] = y_val[:,0]
colors[:,0] = 1-y_val[:,0]

plt.scatter(x_val_noise[:,0],x_val_noise[:,1],c=colors,edgecolors=[0.,0.,1.])
plt.show()

In [ ]:
# we need the strawberry fields plugin for pennylane for this device
# install >> pip install pennylane-sf strawberryfields
dev = qml.device('strawberryfields.fock', wires=2, cutoff_dim=10)

In [ ]:
def layer(v):
    # Matrix multiplication of input layer
    qml.Rotation(v[0], wires=0)
    qml.Squeezing(v[1], 0., wires=0)
    qml.Rotation(v[2], wires=0)
    qml.Rotation(v[0], wires=1)
    qml.Squeezing(v[1], 0., wires=1)
    qml.Rotation(v[2], wires=1)

    # Bias
    qml.Displacement(v[3], 0., wires=0)
    qml.Displacement(v[3], 0., wires=1)

    # Beamsplitter
    qml.Beamsplitter(v[4],v[5],wires = [0,1])
    
    # Element-wise nonlinear transformation
    qml.Kerr(v[6], wires=0)
    qml.Kerr(v[7], wires=1)
    
    
@qml.qnode(dev)
def quantum_neural_net(var, x=None):
    
    # Encode input x into quantum state
    qml.Displacement(x[0], 0., wires=0)
    qml.Displacement(x[1], 0., wires=1)

    # "layer" subcircuits
    for v in var:
        layer(v)
        
    return qml.expval.X(0)
    #return qml.expval.X(1)

def square_loss(labels, predictions):

    loss = 0.
    for l, p in zip(labels, predictions):
        loss += (l - p) ** 2
    loss = loss / len(labels)

    return loss

def cost(var, features, labels):

    preds = [quantum_neural_net(var, x=x) for x in features]

    return square_loss(labels, preds)

In [ ]:
np.random.seed(0)
num_layers = 2
var_init = 0.05 * np.random.randn(num_layers, 8)

var_init

In [ ]:
opt = AdamOptimizer(0.01, beta1=0.9, beta2=0.999)

#var = var_init
for it in range(512,1024):
    #var = opt.step(lambda v: cost(v, x, y), var)
    
    var = opt.step(lambda v: cost(v, x_noise, y), var)
    
    print("Iter: {:5d} | Cost: {:0.7f} ".format(it + 1, cost(var, x_noise, y)[0]))
    
    if it % 20 == 0:
        
        x_val_noise = x_val_noise + np.random.randn(num_samples,2)*1e-3
        predictions = np.round(np.array([quantum_neural_net(var, x=x_) for x_ in x_val_noise]))

        
        colors = np.zeros((num_samples,3))
        colors[:,1] = predictions #- (y[:,0])
        colors[:,0] = 1 - predictions
        
        colors[colors<0] = 0.
        colors[colors>1] = 1.
        
        edge_colors = np.zeros((num_samples,3))
        edge_colors[:,1] = y_val[:,0]
        edge_colors[:,0] = 1. - y_val[:,0]
        
        edge_colors[edge_colors<0] = 0.
        edge_colors[edge_colors>1] = 1.
        
        fig, axes = plt.subplots(1,1,figsize=(10,10))
        
        axes.scatter(x_val_noise[:,0],x_val_noise[:,1],c=colors,edgecolors=edge_colors,lw=2,s=640)
        axes.set_xticks([0,1])
        axes.set_xticklabels(['0','1'],fontsize=48)
        axes.set_yticks([0,1])
        axes.set_yticklabels(['0','1'],fontsize=48)
        plt.title("Noisy XOR",fontsize=56)
        plt.savefig('./noisy_xor%i.png'%it)
        plt.show()

In [ ]:

        
colors = np.zeros((num_samples,3))
colors[:,1] = predictions #- (y[:,0])
colors[:,2] = 1 - predictions

colors[colors<0] = 0.
colors[colors>1] = 1.

edge_colors = np.zeros((num_samples,3))
edge_colors[:,1] = np.squeeze(y_val)
edge_colors[:,2] = np.squeeze(1 - y_val)

edge_colors[edge_colors<0] = 0.
edge_colors[edge_colors>1] = 1.

fig, axes = plt.subplots(1,1,figsize=(10,10))

axes.scatter(x_noise[:,0],x_noise[:,1],c=colors,edgecolors=edge_colors,lw=2,s=640)
axes.set_xticks([0,1])
axes.set_xticklabels(['0','1'],fontsize=48)
axes.set_yticks([0,1])
axes.set_yticklabels(['0','1'],fontsize=48)
plt.title("Noisy XOR",fontsize=56)
plt.savefig('./noisy_xor%i.png'%it)
plt.show()

In [ ]:
y_val

In [ ]:
predictions = np.round(np.array([quantum_neural_net(var, x=x_) for x_ in x_noise]))

colors = np.zeros((num_samples,3))
colors[:,0] = predictions #- (y[:,0])
colors[:,1] = 1- predictions

colors[colors>1.] = 1.
colors[colors<0.] = 0.
plt.scatter(x_noise[:,0],x_noise[:,1],c=colors,edgecolors=[0.,0.,0.])
plt.show()

In [ ]:
print(x,predictions)

# generate noisy XOR data

def xor(x):
    n = x.shape[0]
    
    y = np.zeros((n,1))
    for ck in range(n):
        y[ck] = (x[ck,0] or x[ck,1]) and not(x[ck,0] and x[ck,1])

    return y

def to_onehot(y):
    n = y.shape[0]
    num_classes = int(np.max(y)+1)
    
    one_hot_y = np.zeros((n,num_classes))
    for ck in range(n):
        one_hot_y[ck,int(y[ck])] = 1
    
    return one_hot_y

    
num_samples = 32
x = np.random.randint(0,2,size=(num_samples,2))
x_noise = x + np.random.randn(num_samples,2)*2e-1

y = xor(x)
    
y = to_onehot(y)


num_samples = 32
x_val = np.random.randint(0,2,size=(num_samples,2))
x_noise_val = x + np.random.randn(num_samples,2)*2e-1

y_val = xor(x_val)
y_val = to_onehot(y_val) 